# Function Generation for the Training of λ-Nets

## Package installation (uncommand first line to install packages at the beginning)

In [1]:
%%script false --no-raise-error
#if some errors occur during the installation consider using "sudo pip install"
!pip install numpy
!pip install pandas
!pip install truth-table-generator
!pip install more-itertools
!pip install tqdm
!pip install joblib
!pip install scipy
!pip install PrettyTable
!pip install colored
!pip install scikit-learn
!pip install keras
!pip install ipython
!pip install livelossplot
!pip install matplotlib
!pip install seaborn
!pip install tensorflow
!pip install tensorflow-gpu

## Specitication of Experiment Settings

In [2]:
n = 4  # number of variables

interpretation_dataset_size = 2**(2**(4)) #specifies the number of functions generated (specifies the interpretation-net dataset size), default to 2**(2**(4))


In [3]:
##############DO NOT CHANGE###################
variables = 'abcdefghijklmnopqrstuvwxyz'[:n]

RANDOM_SEED = 42

max_size = True if (interpretation_dataset_size == 2**(2**(4)) and n ==4) else False

print('Variables: ' + str(n) + ' (' + variables + ')')
print('Dataset Size: ' + str(interpretation_dataset_size))

Variables: 4 (abcd)
Dataset Size: 65536


## Imports

In [4]:
import ttg
from itertools import product       # forms cartesian products
from more_itertools import random_product
from tqdm import tqdm_notebook as tqdm
import pickle

import pandas as pd
from joblib import Parallel, delayed
import random 
from random import sample 
random.seed(RANDOM_SEED)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

#generate empty directories needed
directory_names = ['parameters', 'pickled_expression_lists', 'results', 'saved_models', 'weights', 'weights_training']
if not os.path.exists('./data'):
    os.mkdir('./data')
for directory_name in directory_names:
    path = './data/' + directory_name
    if not os.path.exists(path):
        os.mkdir(path)

# Function Generation

In [5]:
list_of_expessions = []

pairs = [('~'+var, var) for var in variables]

inputs = list(product(*pairs))

if max_size:
    for i, outputs in tqdm(enumerate(product([0, 1], repeat=len(inputs)))):    
        terms = [' and '.join(row) for row, output in zip(inputs, outputs) if output]
        if not terms:
            terms = ['False']
        list_of_expessions.append(' or '.join(terms))
else:
    for i in tqdm(range(interpretation_dataset_size)):
        outputs = random_product([0, 1], repeat=len(inputs))
        terms = [' and '.join(row) for row, output in zip(inputs, outputs) if output]
        if not terms:
            terms = ['False']
        expression = ' or '.join(terms)
        while expression in list_of_expessions:
            outputs = random_product([0, 1], repeat=len(inputs))
            terms = [' and '.join(row) for row, output in zip(inputs, outputs) if output]
            if not terms:
                terms = ['False']
            expression = ' or '.join(terms)           
        list_of_expessions.append(expression)
        


In [6]:
list_of_tables = []
for expression in tqdm(list_of_expessions):
    table = ttg.Truths([variable for variable in variables[:n]], [expression])
    list_of_tables.append(table)

In [7]:
def asPandas(table):
    pandas_table = table.as_pandas()
    return pandas_table

list_of_tables_pandas = Parallel(n_jobs=-1, verbose=1, backend='multiprocessing')(map(delayed(asPandas), list_of_tables)) #, backend="threading"



[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 756 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 1456 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 2356 tasks      | elapsed:   36.0s
[Parallel(n_jobs=-1)]: Done 3456 tasks      | elapsed:   54.3s
[Parallel(n_jobs=-1)]: Done 4756 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 6256 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 7626 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 8576 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 9626 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 10776 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 12026 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 13376 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 14

In [8]:
#Generate the zhegalkin polynoms and add them to the dataframe
from sympy import *
from sympy.logic.boolalg import Xor, BooleanTrue
from sympy.core.compatibility import ordered, as_int
from sympy.abc import a, b, c, d, e, f

def anf_coeffs(truthvalues):
    """
    Convert a list of truth values of some boolean expression
    to the list of coefficients of the polynomial mod 2 (exclusive
    disjunction) representing the boolean expression in ANF
    (i.e., the "Zhegalkin polynomial").
    There are 2^n possible Zhegalkin monomials in n variables, since
    each monomial is fully specified by the presence or absence of
    each variable.
    We can enumerate all the monomials. For example, boolean
    function with four variables (a, b, c, d) can contain
    up to 2^4 = 16 monomials. The 13-th monomial is the
    product a & b & d, because 13 in binary is 1, 1, 0, 1.
    A given monomial's presence or absence in a polynomial corresponds
    to that monomial's coefficient being 1 or 0 respectively.
    Examples
    ========
    >>> from sympy.logic.boolalg import anf_coeffs, bool_monomial, Xor
    >>> from sympy.abc import a, b, c
    >>> truthvalues = [0, 1, 1, 0, 0, 1, 0, 1]
    >>> coeffs = anf_coeffs(truthvalues)
    >>> coeffs
    [0, 1, 1, 0, 0, 0, 1, 0]
    >>> polynomial = Xor(*[
    ...     bool_monomial(k, [a, b, c])
    ...     for k, coeff in enumerate(coeffs) if coeff == 1
    ... ])
    >>> polynomial
    b ^ c ^ (a & b)
    """

    s = '{0:b}'.format(len(truthvalues))
    n = len(s) - 1

    if len(truthvalues) != 2**n:
        raise ValueError("The number of truth values must be a power of two, "
                         "got %d" % len(truthvalues))

    coeffs = [[v] for v in truthvalues]

    for i in range(n):
        tmp = []
        for j in range(2 ** (n-i-1)):
            tmp.append(coeffs[2*j] +
                list(map(lambda x, y: x^y, coeffs[2*j], coeffs[2*j+1])))
        coeffs = tmp

    return coeffs[0]

def bool_monomial(k, variables):
    """
    Return the k-th monomial.
    Monomials are numbered by a binary encoding of the presence and
    absences of the variables. This convention assigns the value
    1 to the presence of variable and 0 to the absence of variable.
    Each boolean function can be uniquely represented by a
    Zhegalkin Polynomial (Algebraic Normal Form). The Zhegalkin
    Polynomial of the boolean function with n variables can contain
    up to 2^n monomials. We can enumarate all the monomials.
    Each monomial is fully specified by the presence or absence
    of each variable.
    For example, boolean function with four variables (a, b, c, d)
    can contain up to 2^4 = 16 monomials. The 13-th monomial is the
    product a & b & d, because 13 in binary is 1, 1, 0, 1.
    Parameters
    ==========
    k : int or list of 1's and 0's
    variables : list of variables
    Examples
    ========
    >>> from sympy.logic.boolalg import bool_monomial
    >>> from sympy.abc import x, y, z
    >>> bool_monomial([1, 0, 1], [x, y, z])
    x & z
    >>> bool_monomial(6, [x, y, z])
    x & y
    """
    if isinstance(k, int):
        k = integer_to_term(k, len(variables))
    variables = list(map(sympify, variables))
    return _convert_to_varsANF(k, variables)

def integer_to_term(k, n_bits=None):
    """
    Return a list of the base-2 digits in the integer, ``k``.
    Parameters
    ==========
    k : int
    n_bits : int
        If ``n_bits`` is given and the number of digits in the binary
        representation of ``k`` is smaller than ``n_bits`` then left-pad the
        list with 0s.
    Examples
    ========
    >>> from sympy.logic.boolalg import integer_to_term
    >>> integer_to_term(4)
    [1, 0, 0]
    >>> integer_to_term(4, 6)
    [0, 0, 0, 1, 0, 0]
    """

    s = '{0:0{1}b}'.format(abs(as_int(k)), as_int(abs(n_bits or 0)))
    return list(map(int, s))

def _convert_to_varsANF(term, variables):
    """
    Converts a term in the expansion of a function from binary to it's
    variable form (for ANF).
    Parameters
    ==========
    term : list of 1's and 0's (complementation patter)
    variables : list of variables
    """
    temp = []
    for i, m in enumerate(term):
        if m == 1:
            temp.append(variables[i])
        else:
            pass # ignore 0s

    if temp == []:
        return BooleanTrue()

    return And(*temp)



In [9]:
#Convert to zhegalkin polynoms and append it
truthvalues_list = []
tmp_list = []
for formula in list_of_tables_pandas:
    for row in formula.values:
        tmp_list.append(row[n])
    truthvalues_list.append(tmp_list)
    tmp_list = list()

In [10]:
#Convert truthvalues in Zhegalkin polynomials with the help of the library
var_list = []
for var in variables:
    vars()[var] = None
    var_list.append(var)
    
zhegalkin_polynomial_list = list()
for truthvalues in truthvalues_list:
    coeffs = anf_coeffs(truthvalues)
    polynomial = Xor(*[
             bool_monomial(k, var_list)
             for k, coeff in enumerate(coeffs) if coeff == 1 ])
    zhegalkin_polynomial_list.append(polynomial)        

In [11]:
#Create a coeff to binar representation dictionary
def getPolynomialValues(coeff, table, val_list):
    
    #if the first coefficient is 1, set the first row of the truth table to one
    if coeff[0] == '1':
        val_list[0] = 1
        return val_list
    
    
    cleaned_string_coeff = ''.join([c for c in coeff if c in variables])
    if len(cleaned_string_coeff) <= 1:
        #Take the last index for the value as this is the value that has to be one for a single coefficient
        indexes = table.loc[(table[cleaned_string_coeff] == 1)].index
        index_last = 0
        for ind in indexes:
            #index starts at 0, so -1 is needed
            index_last = ind - 1
        val_list[index_last] = 1
    else:
        for i in range(len(cleaned_string_coeff)):
            table = table.loc[(table[cleaned_string_coeff[i]] == 1)]
        for ind in table.index:
            val_list[ind-1] = 1
    
    return val_list

In [12]:
import re

#Conversion of the sympi Xor, False and Not Zhegalkin representations to string representations that are added
#to the dataframe to identify the polynoms
tmp_polynom = ''
result_polynoms = dict()
result_values = list()


#for conversion purposes we need the values for all variables again
table = ttg.Truths([variable for variable in variables[:n]])
pd_table = asPandas(table)



for formula in zhegalkin_polynomial_list:
    tmp_res = [0 for val in range(pd_table.shape[0])]

    if (formula == False or formula == '0'):
        tmp_polynom += str(0)
        tmp_res = [0 for i in range(pd_table.shape[0])]
    
    elif formula == True:
        tmp_polynom += '1'
        tmp_res = [1 for i in range(pd_table.shape[0])]
   
    elif type(formula) == Xor or type(formula) == And or type(formula) == Symbol:
        string_formula = str(formula)
        
        #Split at comma and whitespace to get the coefficients
        tmp_spl= re.split("\^\s",string_formula)
        
        for coeff in tmp_spl:
            #Filter out ")" at the end of the strings
            if(coeff.endswith(')')):
                tmp_polynom += coeff
                #recursively overwrite tmp_res so that at the end the final value list has only values for the truth table of the polynoms
                tmp_res = getPolynomialValues(coeff[:-1], pd_table, tmp_res)

            else:
                tmp_polynom += coeff + ' xor '
    
                tmp_res = getPolynomialValues(coeff, pd_table, tmp_res)

    
    elif type(formula) == Not:
        tmp_polynom += '1 xor '
        tmp_res = getPolynomialValues('1', pd_table, tmp_res)
        string_formula = str(formula)
        #Split the string and remove Not
        x = re.split("\~\(", string_formula, 1)
        #Split at comma and whitespace to get the coefficients
        if (len(x) == 1):
            tmp_polynom += x[0]
        else:
            tmp_spl= re.split("\^\s", x[1])
            for coeff in tmp_spl:

                #Filter out ") at the end ot the strings
                if(coeff.endswith(')')):
                    tmp_polynom += coeff[:-1]
                    tmp_res = getPolynomialValues(coeff[:-1], pd_table, tmp_res)
                else:
                    tmp_polynom += coeff + ' xor '
                    tmp_res = getPolynomialValues(coeff, pd_table, tmp_res)

    result_polynoms[tmp_polynom] = tmp_res
    tmp_polynom = ''
    result_values = list()

    

In [13]:
#Add it to the dataframe now
counter = 0
for key, value_list in result_polynoms.items():
    list_of_tables_pandas[counter][key] = value_list
    counter += 1

In [14]:
list_of_tables_pandas[3]

,a,b,c,d,a and b and c and ~d or a and b and c and d,1 xor a xor b xor c xor (a & b) xor (a & c) xor (b & c) xor (a & b & c)
1,1,1,1,1,1,1
2,1,1,1,0,1,1
3,1,1,0,1,0,1
4,1,1,0,0,0,1
5,1,0,1,1,0,1
6,1,0,1,0,0,1
7,1,0,0,1,0,0
8,1,0,0,0,0,1
9,0,1,1,1,0,1
10,0,1,1,0,0,1


In [15]:
if max_size:
    path = './data/pickled_expression_lists/total_expressions_length_' + str(n) + '_variables_' + str(n) + '_df.pkl'
else:
    path = './data/pickled_expression_lists/sample' + str(interpretation_dataset_size) +  '_expressions_length_' + str(n) + '_variables_' + str(n) + '_df.pkl'
with open(path, 'wb') as f:
    pickle.dump(list_of_tables_pandas, f, protocol=2)
    